In [20]:
#| label: imports-libraries
#| echo: false
#| output: false
#| include: true

## Data manipulation imports
import pandas as pd
import numpy as np

## Display imports
from IPython.display import display, Markdown

## statmodels import
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.genmod.families.family as fam
from patsy import dmatrices

## Image extractor
from pypdf import PdfReader

## Plot imports
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (5,5/2.5)
import seaborn as sns
sns.set_style('whitegrid')
sns.set_theme()
sns.set_context(
    "paper", 
    rc={
        "figsize"       :   plt.rcParams['figure.figsize'],
        'font_scale'    :   1.25,
    }
)

def extract_images_from_pdf(
        output_path : str = "../imgs/",
        pdf_path : str = "../data/pdf/DC_airflow_chapter1.pdf",
        page_number : int = -5,
    ) -> None:
    output_path = pdf_path.split('/')[-1].split('_')[-1].split('.')[0]
    print(output_path)
    ## Create an instance of PdfReader with the PDF path
    reader = PdfReader(pdf_path)
    ## Define the page to extract images from
    page = reader.pages[page_number]
    ## Create a counter to name the images
    count = 0

    ## Loop through the images in the page
    for image_file_object in page.images:
        with open('../imgs/' + output_path + '/' + str(count) + image_file_object.name, "wb") as fp:
            fp.write(image_file_object.data)
            count += 1
    
    return None

In [22]:
#| label: export-images
#| echo: false
#| output: false
#| include: true

[
    extract_images_from_pdf(
        pdf_path = "../data/pdf/DC_airflow_chapter2.pdf",
        page_number = page,
    )
    for page in range(24,33,1)
]

chapter2
chapter2
chapter2
chapter2
chapter2
chapter2
chapter2
chapter2
chapter2


[None, None, None, None, None, None, None, None, None]

# Section 1: Introduction to Airflow
Apache Airflow is a platform to program workflows (general), including the creation, scheduling, and monitoring of said workflows. Airflow can use various tools and languages, but the actual workflow code is written with Python. Airflow implements workflows as DAGs, or Directed Acyclic Graphs. Airflow can be accessed and controlled via code, via the command-line, or via a built-in web interface. 

## What is a workflow?

Before we can really discuss Airflow, we need to talk about workflows. A workflow is a set of steps to accomplish a given data engineering task. These can include any given task, such as downloading a file, copying data, filtering information, writing to a database, and so forth. A workflow is of varying levels of complexity. Some workflows may only have 2 or 3 steps, while others consist of hundreds of components. 

![Workflow example containing 5 steps](../tutorials/0X20.png){width=40%}

The complexity of a workflow is completely dependent on the needs of the user. We show an example of a possible workflow to the right. It's important to note that we're defining a workflow here in a general data engineering sense. This is an informal definition to introduce the concept. As you'll see later, workflow can have specific meaning within specific tools.

## What is Airflow?

Airflow is a platform to program workflows (general), including the creation, scheduling, and monitoring of said workflows.

Airflow can use various tools and languages, but the actual workflow code is written with Python. Airflow implements workflows as DAGs, or Directed Acyclic Graphs. We'll discuss exactly what this means throughout this course, but for now think of it as a set of tasks and the dependencies between them. Airflow can be accessed and controlled via code, via the command-line, or via a built-in web interface. We'll look at all these options later on.

## Quick introduction to DAGs

A DAG stands for a Directed Acyclic Graph. In Airflow, this represents the set of tasks that make up your workflow. It consists of the tasks and the dependencies between tasks. 

![Example of DAG consisting of five set of tasks](../tutorials/0X48.png)

DAGs are created with various details about the DAG, including the name, start date, owner, email alerting options, etc.

## DAG code example

We will go into further detail in the next lesson but a very simple DAG is defined using the following code. A new DAG is created with the dag_id of etl_pipeline and a default_args dictionary containing a start_date for the DAG. 

```python
etl_dag = DAG(
    dag_id = 'etl_pipeline',    
    default_args = {"start_date": "2020-01-08"},
)
```
::: {.callout-caution collapse="true" apperance='simple'} 
Note that within any Python code, this is referred to via the variable identifier, etl_dag, but within the Airflow shell command, you must use the dag_id.
:::

## Running a workflow in Airflow

To get started, let's look at how to run a component of an Airflow workflow. These components are called tasks and simply represent a portion of the workflow. We'll go into further detail in later chapters. There are several ways to run a task, but one of the simplest is using the airflow run shell command. 

```bash
airflow run <dag_id> <task_id> <start_date>
```

Airflow run takes three arguments, a dag_id, a task_id, and a start_date. All of these arguments have specific meaning and will make more sense later in the course. For our example, we'll use a dag_id of example-etl, a task named download-file, and a start date of 2020-01-10. This task would simply download a specific file, perhaps a daily update from a remote source. Our command as such is airflow run example-etl download-file 2020-01-10. This will then run the specified task within Airflow.

## Exercises
We've looked at Airflow and some of the basic aspects of why you'd use it. We've also looked at how to run a task within Airflow from the command-line. Let's practice what we've learned.

### Exercise 1: Running a task in Airflow
You've just started looking at using Airflow within your company and would like to try to run a task within the Airflow platform. You remember that you can use the `airflow run` command to execute a specific task within a workflow. 

::: {.callout-caution collapse="true" apperance='simple'}  
Note that an error while using `airflow run` will return `airflow.exceptions.AirflowException:` on the last line of output.
::: 

An Airflow DAG is set up for you with a dag_id of `etl_pipeline`. The `task_id` is `download_file` and the `start_date` is `2020-01-08`. All other components needed are defined for you.

Which command would you enter in the console to run the desired task?

- [ ] airflow run dag task 2020-01-08  
- [ ] airflow run etl_pipeline task 2020-01-08  
- [ ] airflow run etl_pipeline download_file 2020-01-08  

### Exercise 2: Examining Airflow commands
While researching how to use Airflow, you start to wonder about the airflow command in general. You realize that by simply running airflow you can get further information about various sub-commands that are available.

Which of the following is NOT an Airflow sub-command?

- [ ] list_dags
- [ ] edit_dag
- [ ] test
- [ ] scheduler

# Section 2: Airflow DAGs
Our first question is what is a DAG? Beyond any specific mathematical meaning, a DAG, or Directed Acyclic Graph, has the following attributes: 

- It is **Directed**, meaning there is an inherent flow representing the dependencies or order between execution of components. These dependencies (even implicit ones) provide context to the tools on how to order the running of components. 

- A DAG is also **Acyclic** - it does not loop or repeat. This does not imply that the entire DAG cannot be rerun, only that the individual components are executed once per run. 

- In this case, a **Graph** represents the components and the relationships (or dependencies) between them. 

The term DAG is found often in data engineering, not just in Airflow but also Apache Spark, Luigi, and others^[https://en.m.wikipedia.org/wiki/Directed_acyclic_graph].

![](../imgs/0X78.png)

## DAG in Airflow

As we're working with Airflow, let's look at its implementation of the DAG concept. Within Airflow, DAGs are written in Python, but can use components written in other languages or technologies. This means we'll define the DAG using Python, but we could include Bash scripts, other executables, Spark jobs, and so on. Airflow DAGs are made up of components to be executed, such as operators, sensors, etc. Airflow typically refers to these as tasks. We'll cover these in much greater depth later on, but for now think of a task as a thing within the workflow that needs to be done. Airflow DAGs contain dependencies that are defined, either explicitly or implicitly. These dependencies define the execution order so Airflow knows which components should be run at what point within the workflow. For example, you would likely want to copy a file to a server prior to trying to import it to a database.

## Define a DAG

Let's look at defining a simple DAG within Airflow. When defining the DAG in Python, you must first import the DAG object from airflow dot models. Once imported, we create a default arguments dictionary consisting of attributes that will be applied to the components of our DAG. These attributes are optional, but provide a lot of power to define the runtime behavior of Airflow. 

```python
from airflow.models import DAG
from datetime import datetime

default_arguments = {
    'owner'         : 'jdoe',
    'email'         : 'jdoe@email.com', 
    'start_date'    : datetime(2020, 1, 20)
}

etl_dag = DAG(
    'etl_workflow', 
    default_args = default_arguments
)
```

Here we define the owner name as jdoe, an email address for any alerting, and specify the start date of the DAG. The start date represents the earliest datetime that a DAG could be run. Finally, we define our DAG object with the first argument using a name for the DAG, etl underscore workflow, and assign the default arguments dictionary to the default underscore args argument. There are many other optional configurations we will use later on. 

::: {.callout-caution collapse="true" apperance='simple'}  
Note that the entire DAG is assigned to a variable called etl underscore dag. This will be used later when defining the components of the DAG, but the variable name etl underscore dag does not actually appear in the Airflow interfaces. Note, DAG is case sensitive in Python code.
::: 

## DAGs on the command line

When working with DAGs (and Airflow in general), you'll often want to use the airflow command line tool. The airflow command line program contains many subcommands that handle various aspects of running Airflow. You've used a couple of these already in previous exercises. Use the 

```bash
airflow -h
```

command for help and descriptions of the subcommands. Many of these subcommands are related to DAGs. You can use the 

```bash
airflow list_dags
```

option to see all recognized DAGs in an installation. When in doubt, try a few different commands to find the information you're looking for.

## Command line vs Python

You may be wondering when to use the Airflow command line tool vs writing Python. 

|        Command line        |           Python           |
|----------------------------|----------------------------|
| Start Airflow processes    | Create a DAGs              |
| Manually run DAGs / tasks  | Edit individual prop of DAG |
| Review logging information |                            |
: Command line vs Python


In general, the airflow command line program is used to start Airflow processes (ie, webserver or scheduler), manually run DAGs or tasks, and review logging information. Python code itself is usually used in the creation and editing of a DAG, not to mention the actual data processing code itself.

## Exercises
### Exercise 1: Defining a simple DAG
You've spent some time reviewing the Airflow components and are interested in testing out your own workflows. To start you decide to define the default arguments and create a DAG object for your workflow.

The DateTime object has been imported for you.

1. Import the Airflow DAG object. Note that it is case-sensitive.
2. Define the default_args dictionary with a key owner and a value of 'dsmith'. Add a start_date of January 14, 2020 to default_args using the value 1 for the month of January. Add a retries count of 2 to default_args.
3. Instantiate the DAG object to a variable called etl_dag with a DAG named example_etl. Add the default_args dictionary to the appropriate argument.

In [28]:
# Import the DAG object
from datetime import datetime
from airflow.models import DAG

# Define the default_args dictionary
default_args = {
  'owner'       : 'dsmith',
  'start_date'  : datetime(2020, 1, 14),
  'retries'     : 2
}

# Instantiate the DAG object
etl_dag = DAG(
    'example_etl', 
    default_args=default_args
)

### Exercise 2: Working with DAGs and the Airflow shell
While working with Airflow, sometimes it can be tricky to remember what DAGs are defined and what they do. You want to gain some further knowledge of the Airflow shell command so you'd like to see what options are available.

Multiple DAGs are already defined for you. How many DAGs are present in the Airflow system from the command-line?

In [1]:
!airflow dags list

/Users/bernardo/miniforge3/envs/blog/lib/python3.11/site-packages/airflow/configuration.py:751 UserWarning: Config scheduler.max_tis_per_query (value: 512) should NOT be greater than core.parallelism (value: 32). Will now use core.parallelism as the max task instances per query instead of specified value.

Please confirm database upgrade (or wait 4 seconds to skip it). Are you sure? [y/N]
No data found


# Section 3: Airflow web interface

The Airflow web UI is made up of several primary page groups useful in developing and administering workflows on the Airflow platform. 

::: {.callout-caution collapse="true" apperance='simple'} 
Note  that for this course, we'll only be focusing on a few pages but it's helpful to click around the various options and get familiar with what's available. The DAGs view of the Airflow UI is the page we'll spend most of our time.
::: 

## DAGs view DAGs

It provides a quick status of the number of DAGs / workflows available.

![](../imgs/0X117.png)

![](../imgs/0X121.png)

It shows us the schedule for the DAG (in date or cron format).

![](../imgs/0X125.png)

We can see the owner of the DAG.

![](../imgs/0X129.png)

which of the most recent tasks have run,

![](../imgs/0X133.png)

when the last run started,

![](../imgs/0X137.png)

and the last three DAG runs.

![](../imgs/0X141.png)

The links area on the right gives us quick access to many of the DAG specific views.

![](../imgs/0X145.png)

Don't worry about those for now - instead we'll click on the "example_dag" link which takes us to our DAG detail page.

![](../imgs/0X149.png)

## DAG detail view

The DAG detail view gives us specific access to information about the DAG itself, including several views of information (Graph, Tree, and Code) illustrating the tasks and dependencies in the code. We also get access to the Task duration, task tries, timings, a Gantt chart view, and specific details about the DAG. We have the ability to trigger the DAG (to start), refresh our view, and delete the DAG if we desire. The detail view defaults to the Tree view, showing the specific named tasks, which operators are in use, and any dependencies between tasks. The circles in front of the words represent the state of the task / DAG. In the case of our specific DAG, we see that we have one task called generate_random_number.

![](../imgs/0X153.png)

## DAG graph view

The DAG graph view arranges the tasks and dependencies in a chart format - this provides another view into the flow of the DAG. You can see the operators in use and the state of the tasks at any point in time. The tree and graph view provide different information depending on what you'd like to know. Try moving between them when examining a DAG to obtain further details. For this view we again see that we have a task called generate_random_number. We can also see that it is of the type BashOperator in the middle left of the image.

![](../imgs/0X159.png)

## DAG code view

The DAG code view does exactly as it sounds - it provides a copy of the Python code that makes up the DAG. The code view provides easy access to exactly what defines the DAG without clicking in various portions of the UI. As you use Airflow, you'll determine which tools work best for you. It is worth noting that the code view is read-only. Any DAG code changes must be done via the actual DAG script. In this view, we can finally see the code making up the generate_random_number task and that it runs the bash command echo $RANDOM.

![](../imgs/0X165.png)

## Logs

The Logs page, under the Browse menu option, provides troubleshooting and audit ability while using Airflow. This includes items such as starting the Airflow webserver, viewing the graph or tree nodes, creating users, starting DAGs, etc. When using Airflow, look at the logs often to become more familiar with the types of information included, and also what happens behind the scenes of an Airflow install. 

::: {.callout-caution collapse="true" apperance='simple'} 
Note  that you'll often refer to the Event type present on the Logs view when searching (such as graph, tree, cli scheduler).
::: 

![](../imgs/0X171.png)

## Web UI vs command line

In most circumstances, you can choose between using the Airflow web UI or the command line tool based on your preference. The web UI is often easier to use overall. The command line tool may be simpler to access depending on settings (via SSH, etc.)

## Exercises
### Exercise 1: Examining DAGs with the Airflow UI
You've become familiar with the basics of an Airflow DAG and the basics of interacting with Airflow on the command-line. Your boss would like you to show others on your team how to examine any available DAGs. In this instance, she would like to know which operator is NOT in use with the DAG called update_state, as your team is trying to verify the components used in production workflows.

Remember that the Airflow UI allows various methods to view the state of DAGs. The Tree View lists the tasks and any ordering between them in a tree structure, with the ability to compress / expand the nodes. The Graph View shows any tasks and their dependencies in a graph structure, along with the ability to access further details about task runs. The Code view provides full access to the Python code that makes up the DAG.

Remember to select the operator NOT used in this DAG.

![](../imgs/Exercise_3_1.png)

- [ ] BashOperator
- [ ] PythonOperator
- [ ] JdbcOperator
- [ ] SimpleHttpOperator

# Section 4: Airflow operators
Airflow operators represent a single task in a workflow. This can be any type of task from running a command, sending an email, running a Python script, and so on. Typically Airflow operators run independently - meaning that all resources needed to complete the task are contained within the operator. Generally, Airflow operators do not share information between each other. This is to simplify workflows and allow Airflow to run the tasks in the most efficient manner. It is possible to share information between operators, but the details of how are beyond this course. Airflow contains many various operators to perform different tasks. For example, the DummyOperator can be used to represent a task for troubleshooting or a task that has not yet been implemented. 

```python
## Arguments to be passed to each operator
default_args = {
    'owner'         : 'jdoe',
    'email'         : 'jdoe@email.com',
    'start_date'    : datetime(2020, 1, 20)
}

## Create a DAG object
dag = DAG(
    'example_dag', 
    default_args = default_args
)

## Define tasks/operators
DummyOperator(
    task_id = 'dummy_task', 
    dag = dag
)
```
We are focusing on the BashOperator for this lesson but will look at the PythonOperator and several others later on.


## BashOperator
The BashOperator executes a given Bash command or script. 

```python
BashOperator(    
    task_id='bash_example',    
    bash_command='echo "Example!"',    
    dag=ml_dag
)

BashOperator(    
    task_id='bash_script_example',    
    bash_command='runcleanup.sh',    
    dag=ml_dag
)
`````

This command can be pretty much anything Bash is capable of that would make sense in a given workflow. The BashOperator requires three arguments: the task id which is the name that shows up in the UI, the bash command (the raw command or script), and the dag it belongs to. The BashOperator runs the command in a temporary directory that gets automatically cleaned up afterwards. It is possible to specify environment variables for the bash command to try to replicate running the task as you would on a local system. If you're unfamiliar with environment variables, these are run-time settings interpreted by the shell. It provides flexibility while running scripts in a generalized way. The first example runs a bash command to echo Example exclamation mark to standard out. The second example uses a predefined bash script for its command, runcleanup.sh.


## BashOperator examples

Before using the BashOperator, it must be imported: 

```python
from airflow.operators.bash_operator import BashOperator

...

example_task = BashOperator(
    task_id = 'example',
    bash_command = 'echo 1',
    dag = dag,
)
```

The first example creates a BashOperator that takes a task_id, runs the bash command "echo 1", and assigns the operator to the dag. 
::: {.callout-caution collapse="true" apperance='simple'} 
Note that we've previously defined the dag in an earlier exercise.
::: 

The second example is a BashOperator to run a quick data cleaning operation using cat and awk.
```python
bash_task = BashOperator(
    task_id = 'clean_addresses',
    bash_command = 'cat addresses.txt | awk "NF==10" > cleaned.txt',
    dag = dag,
)
```

Don't worry if you don't understand exactly what this is doing. This is a common scenario when running workflows - you may not know exactly what a command does, but you can still run it in a reliable way.


## Operator gotchas

There are some general gotchas when using Operators. The biggest is that individual operators are not guaranteed to run in the same location or environment. This means that just because one operator ran in a given directory with a certain setup, it does not necessarily mean that the next operator will have access to that same information. If this is required, you must explicitly set it up. You may need to set up environment variables, especially for the BashOperator. 

> For example, it's common in bash to use the tilde character to represent a home directory. This is not defined by default in Airflow. 

> Another example of an environment variable could be AWS credentials, database connectivity details, or other information specific to running a script. 

Finally, it can also be tricky to run tasks with any form of elevated privilege. This means that any access to resources must be setup for the specific user running the tasks. If you're uncertain what elevated privileges are, think of running a command as root or the administrator on a system.

## Exercises
### Exericse 1: Defining a BashOperator task
The BashOperator allows you to specify any given Shell command or script and add it to an Airflow workflow. This can be a great start to implementing Airflow in your environment.

As such, you've been running some scripts manually to clean data (using a script called cleanup.sh) prior to delivery to your colleagues in the Data Analytics group. As you get more of these tasks assigned, you've realized it's becoming difficult to keep up with running everything manually, much less dealing with errors or retries. You'd like to implement a simple script as an Airflow operator.

The Airflow DAG analytics_dag is already defined for you and has the appropriate configurations in place.

- Import the BashOperator object.
- Define a BashOperator called cleanup with the task_id of cleanup_task.
- Use the command cleanup.sh.
- Add the operator to the DAG.

```python
# Import the BashOperator
from airflow.operators.bash_operator import BashOperator

# Define the BashOperator 
cleanup = BashOperator(
    task_id='cleanup_task',
    # Define the bash_command
    bash_command='cleanup.sh',
    # Add the task to the dag
    dag=analytics_dag,
)
```

### Exercise 2: Multiple BashOperators
Airflow DAGs can contain many operators, each performing their defined tasks.

You've successfully implemented one of your scripts as an Airflow task and have decided to continue migrating your individual scripts to a full Airflow DAG. You now want to add more components to the workflow. In addition to the cleanup.sh used in the previous exercise you have two more scripts, consolidate_data.sh and push_data.sh. These further process your data and copy to its final location.

The DAG analytics_dag is available as before, and your cleanup task is still defined. The BashOperator is already imported.

- Define a `BashOperator` called `consolidate`, to run `consolidate_data.sh` with a `task_id` of `consolidate_task`.
- Add a final `BashOperator` called `push_data`, running `push_data.sh` and a `task_id` of `pushdata_task`.

```python
# Define a second operator to run the `consolidate_data.sh` script
consolidate = BashOperator(
    task_id = 'consolidate_task',
    bash_command = 'consolidate_data.sh',
    dag = analytics_dag,
)

# Define a final operator to execute the `push_data.sh` script
push_data = BashOperator(
    task_id = 'pushdata_task',
    bash_command = 'push_data.sh',
    dag = analytics_dag,
)
```

# Section 5: Airflow tasks

Within Airflow, tasks are instantiated operators. It basically is a shortcut to refer to a given operator within a workflow. Tasks are usually assigned to a variable within Python code. Using a previous example, we assign the BashOperator to the variable example underscore task. 

```python
example_task = BashOperator(
    task_id = 'bash_example',
    bash_command = 'echo "Example!"',
    dag = dag,
)
```

::: {.callout-caution collapse="true" apperance='simple'} 
Note that within the Airflow tools, this task is referred by its task id, not the variable name.
::: 


## Task dependencies

Task dependencies in Airflow define an order of task completion. While not required, task dependencies are usually present. If task dependencies are not defined, task execution is handled by Airflow itself with no guarantees of order. Task dependencies are referred to as upstream or downstream tasks. An upstream task means that it must complete prior to any downstream tasks. Since Airflow 1.8, task dependencies are defined using the bitshift operators. The upstream operator is two greater-than symbols. The downstream operator is two less-than symbols.


## Upstream vs Downstream

It's easy to get confused on when to use an upstream or downstream operator. The simplest analogy is that upstream means before and downstream means after. This means that any upstream tasks would need to complete prior to any downstream ones.

## Simple task dependency

Let's look at a simple example involving two bash operators. We define our first task, and assign it to the variable task1. We then create our second task and assign it to the variable task2. Once each operator is defined and assigned to a variable, we can define the task order using the bitshift operators. In this case, we want to run task1 before task2. 

```python
# Define the tasks
task1 = BashOperator(
    task_id='first_task',                     
    bash_command='echo 1',                     
    dag=example_dag)

task2 = BashOperator(
    task_id='second_task',                     
    bash_command='echo 2',                     
    dag=example_dag)
    
# Set first_task to run before second_task
task1 >> task2   # or task2 << task1
```

The most readable method for this is using the upstream operator, two greater-than symbols, as task1 upstream operator task2. 
::: {.callout-caution collapse="true" apperance='simple'} 
Note that you could also define this in reverse using the downstream operator to accomplish the same thing. In this case, it'd be task2 two less-than symbols task1.
::: 

## Task dependencies in the Airflow UI

Let's take a look at what the Airflow UI shows for tasks and their dependencies. In this case, we're looking at the graph view within the Airflow web interface. 
::: {.callout-caution collapse="true" apperance='simple'} 
Note that in the task area, our two tasks, first_task and second_task, are both present, but there is no order to the task execution. This is the DAG prior to setting the task dependency using the bitshift operator.
::: 

![](../imgs/0X75.png)

Now let's look again at the view with a defined order via the bitshift operators. The view is similar but we can see the order of tasks indicated by the directed arrow between first underscore task and second underscore task.

![](../imgs/0X79.png)

## Multiple dependencies

Dependencies can be as complex as required to define the workflow to your needs. We can chain a dependency, in this case setting task1 upstream of task2 upstream of task3 upstream of task4. The Airflow graph view shows a dependency view indicating this order. 

```python
task1 >> task2 >> task3 >> task4
```
![](../imgs/0X85.png)

You can also mix upstream and downstream bitshift operators in the same workflow. If we define task1 upstream of task2 then downstream of task3, we get a configuration different than what we might expect. 

```python
task1 >> task2 << task3

## or
task1 >> task2
task3 >> task2
```

![](../imgs/1X86.png)

This creates a DAG where first underscore task and third underscore task must finish prior to second underscore task. This means we could define the same dependency graph on two lines, in a possibly clearer form. task1 upstream of task2. task3 upstream of task2. 

::: {.callout-caution collapse="true" apperance='simple'} 
Note that because we don't require it, either task1 or task3 could run first depending on Airflow's scheduling.
::: 

## Exercises
### Exercise 1: Define order of BashOperators
Now that you've learned about the bitshift operators, it's time to modify your workflow to include a pull step and to include the task ordering. You have three currently defined components, cleanup, consolidate, and push_data.

The DAG analytics_dag is available as before and the BashOperator is already imported.

- Define a BashOperator called pull_sales with a bash command of `wget https://salestracking/latestinfo?json`.
- Set the pull_sales operator to run before the cleanup task.
- Configure consolidate to run next, using the downstream operator.
- Set push_data to run last using either bitshift operator.

```python
# Define a new pull_sales task
pull_sales = BashOperator(
    task_id = 'pullsales_task',
     bash_command = 'wget https://salestracking/latestinfo?json',
    dag = analytics_dag,
)

# Set pull_sales to run prior to cleanup
pull_sales >> cleanup

# Configure consolidate to run after cleanup
cleanup >> consolidate

# Set push_data to run last
consolidate >> push_data
```

### Exercise 2: Determining the order of tasks
While looking through a colleague's workflow definition, you're trying to decipher exactly in which order the defined tasks run. The code in question shows the following:

```python
pull_data << initialize_process
pull_data >> clean >> run_ml_pipeline
generate_reports << run_ml_pipeline
```

Order the tasks in the sequence defined by the bitshift code, with the first task to run on top and the last task to run on the bottom.

1. initialize_process
1. pull_data
1. clean
1. run_ml_pipeline
1. generate_reports


### Exercise 3: Troubleshooting DAG dependencies
You've created a DAG with intended dependencies based on your workflow but for some reason Airflow won't load / execute the DAG. Try using the terminal to:

- List the DAGs.
- Decipher the error message.
- Use cat workspace/dags/codependent.py to view the Python code.
- Determine which of the following lines should be removed from the Python code. You may want to consider the last line of the file.

![](../imgs/Exercise_5_3.png)

- [ ] task1 >> task2
- [ ] task2 >> task3
- [ ] task3 >> task1

# Section 6: Additional Airflow operators

## PythonOperator

The PythonOperator is similar to the BashOperator, except that it runs a Python function or callable method. Much like the BashOperator, it requires a taskid, a dag entry, and most importantly a python underscore callable argument set to the name of the function in question. You can also pass arguments or keyword style arguments into the Python callable as needed. Our first example shows a simple printme function that writes a message to the task logs. 

```python
from airflow.operators.python_operator import PythonOperator

def printme():
    print("This goes in the logs!")

python_task = PythonOperator(
    task_id = 'simple_print',
    python_callable = printme,
    dag = example_dag,
)
```

We must first import the PythonOperator from the airflow dot operators dot python underscore operator library. Afterwards, we create our function printme, which will write a quick log message when run. Once defined, we create the PythonOperator instance called python underscore task and add the necessary arguments.

## Arguments

The PythonOperator supports adding arguments to a given task. This allows you to pass arguments that can then be passed to the Python function assigned to python callable. The PythonOperator supports both positional and keyword style arguments as options to the task. 
> For this course, we'll focus on using keyword arguments only for the sake of clarity. 

To implement keyword arguments with the PythonOperator, we define an argument on the task called `op_kwargs`. This is a dictionary consisting of the named arguments for the intended Python function.

## op_kwargs example

Let's create a new function called sleep, which takes a length of time argument. It uses this argument to call the time dot sleep method. Once defined, we create a new task called sleep underscore task, with the taskid, dag, and python callable arguments added as before. This time we'll add our op underscore kwargs dictionary with the length of time variable and the value of 5. 

```python
## Function to sleep (needs argument to be passed)
def sleep(length_of_time):
    time.sleep(length_of_time)

sleep_task = PythonOperator(
    task_id = 'sleep',
    python_callable = sleep,
    ## Pass argument to callable
    op_kwargs = {'length_of_time': 5},
    dag = example_dag,
)
```

::: {.callout-caution collapse="true" apperance='simple'} 
Note that the dictionary key must match the name of the function argument. If the dictionary contains an unexpected key, it will be passed to the Python function and typically cause an unexpected keyword argument error.
::: 


## EmailOperator

There are many other operators available within the Airflow ecosystem. The primary operators are in the `airflow.operators` or `airflow.contrib.operators` libraries. 

Another useful operator is the EmailOperator, which as expected sends an email from within an Airflow task. It can contain the typical components of an email, including HTML content and attachments. 
::: {.callout-caution collapse="true" apperance='simple'} 
Note that the Airflow system must be configured with the email server details to successfully send a message.
::: 

## EmailOperator example

A quick example for sending an email would be sending a generated sales report upon completion of a workflow. We first must import the EmailOperator object from airflow dot operators dot email underscore operator. We can then create our EmailOperator instance with the task id, the to, subject, and content fields and a list of any files to attach. 

```python
from airflow.operators.email_operator import EmailOperator

email_task = EmailOperator(
    task_id = 'email_sales_report',
    to = 'sales_manager@example.com',
    subject = 'Automated Sales Report',
    html_content = 'Attached is the latest sales report',
    files = 'latest_sales.xlsx',
    dag = example_dag
)

```

::: {.callout-caution collapse="true" apperance='simple'} Note that in this case we assume the file latest underscore sales dot xlsx was previously generated - later in the course we'll see how to verify that first. Finally we add it to our dag as usual.

## Exercises
### Exercise 1: Using the PythonOperator
You've implemented several Airflow tasks using the BashOperator but realize that a couple of specific tasks would be better implemented using Python. You'll implement a task to download and save a file to the system within Airflow.

The requests library is imported for you, and the DAG process_sales_dag is already defined.

- Define a function called pull_file with two parameters, URL and savepath.
- Use the print() function and Python f-strings to write a message to the logs.
- Import the necessary object to use the Python Operator.
- Create a new task assigned to the variable pull_file_task, with the id pull_file.
- Add the pull_file(URL, savepath) function defined previously to the operator.
- Define the arguments needed for the task.

```python
# Define the method
def pull_file(URL, savepath):
    r = requests.get(URL)
    with open(savepath, 'wb') as f:
        f.write(r.content)    
    # Use the print method for logging
    print(f"File pulled from {URL} and saved to {savepath}")

## Import the PythonOperator
from airflow.operators.python_operator import PythonOperator

# Create the task
pull_file_task = PythonOperator(
    task_id = 'pull_file',
    # Add the callable
    python_callable = pull_file,
    # Define the arguments
    op_kwargs = {
        'URL' : 'http://dataserver/sales.json', 
        'savepath' : 'latestsales.json',
    },
    dag = process_sales_dag
)
```

### Exercise 2: More PythonOperators
To continue implementing your workflow, you need to add another step to parse and save the changes of the downloaded file. The DAG `process_sales_dag` is defined and has the `pull_file` task already added. In this case, the Python function is already defined for you, `parse_file(inputfile, outputfile)`.

Note that often when implementing Airflow tasks, you won't necessarily understand the individual steps given to you. As long as you understand how to wrap the steps within Airflow's structure, you'll be able to implement a desired workflow.

- Define the Python task to the variable parse_file_task with the id parse_file.
- Add the parse_file(inputfile, outputfile) to the Operator.
- Define the arguments to pass to the callable.
- Add the task to the DAG.



```python
# Add another Python task
parse_file_task = PythonOperator(
    task_id='parse_file',
    # Set the function to call
    python_callable = parse_file,
    # Add the arguments
    op_kwargs = {
        'inputfile':'latestsales.json', 
        'outputfile':'parsedfile.json',
    },
    # Add the DAG
    dag = process_sales_dag,
)
    
```

### Exercise 3: EmailOperator and dependencies
Now that you've successfully defined the PythonOperators for your workflow, your manager would like to receive a copy of the parsed JSON file via email when the workflow completes. The previous tasks are still defined and the DAG `process_sales_dag` is configured.

- Import the class to send emails.
- Define the Operator and add the appropriate arguments (to, subject, files).
- Set the task order so the tasks run sequentially (Pull the file, parse the file, then email your manager).

```python
# Import the Operator
from airflow.operators.email_operator import EmailOperator

# Define the task
email_manager_task = EmailOperator(
    task_id = 'email_manager',
    to = 'manager@datacamp.com',
    subject = 'Latest sales JSON',
    html_content = 'Attached is the latest sales JSON file as requested.',
    files = 'parsedfile.json',
    dag = process_sales_dag,
)

# Set the order of tasks
pull_file_task >> parse_file_task >> email_manager_task
```

# Section 7: Airflow Scheduling
When referring to scheduling in Airflow^[https://airflow.apache.org/docs/stable/scheduler.html], we must first discuss a DAG run. This is an instance of a workflow at a given point in time. For example, it could be the currently running instance, or it could be one run last Tuesday at 3pm. A DAG can be run manually, or via the schedule interval parameter passed when the DAG is defined. Each DAG run maintains a state for itself and the tasks within. The DAGs can the following states:

::: {.callout-note appearance="simple"}
running
:::
::: {.callout-important appearance="simple"}
failed
:::
::: {.callout-tip appearance="simple"}
success 
:::

The individual tasks can have these states or others as well (ie, queued, skipped).

## DAG Runs view

Within the Airflow UI, you can view all DAG runs under the Browse: DAG Runs menu option. This provides the assorted details about any DAGs that have run within the current Airflow instance.

![](../imgs/chapter2/0X146.png)

As mentioned, you can view the state of a DAG run within this page, illustrating whether the DAG run was successful or not.

![](../imgs/chapter2/0X150.png)


## Schedule details

When scheduling a DAG, there are many attributes to consider depending on your scheduling needs. The `start_date` value specifies the first time the DAG could be scheduled. This is typically defined with a Python datetime object. The `end_date` represents the last possible time to schedule the DAG. `max_tries` represents how many times to retry before fully failing the DAG run. The `schedule_interval` represents how often to schedule the DAG for execution. There are many nuances to this which we'll cover in a moment.

## Schedule interval

The schedule interval represents how often to schedule the DAG runs. The scheduling occurs between the start date and the potential end date. Note that this is not when the DAGs will absolutely run, but rather a minimum and maximum value of when they could be scheduled. The schedule interval can be defined by a couple methods - with a cron style syntax or via built-in presets.

## cron syntax

The cron syntax is the same as the format for scheduling jobs using the Unix cron tool. 

![](../imgs/chapter2/0X164.png)

It consists of five fields separated by a space, starting with the minute value (0 through 59), the hour (0 through 23), the day of the month (1 through 31), the month (1 through 12), and the day of week (0 through 6). An asterisk in any of the fields represents running for every interval (for example, an asterisk in the minute field means run every minute) A list of values can be given on a field via comma separated values.

## cron examples

```python
0 12 * * *              # Run daily at Noon (12:00)
* * 25 2 *              # Run once per minute, but only on February 25th
0,15,30,45 * * * *      # Run every 15 minutes
```
The cron entry 0 12 asterisk asterisk asterisk means run daily at Noon (12:00) asterisk asterisk 25 2 asterisk represents running once per minute, but only on February 25th. 0 comma 15 comma 30 comma 45 asterisk asterisk asterisk asterisk means to run every 15 minutes.


## Airflow scheduler presets

Airflow has several presets, or shortcut syntax options representing often used time intervals. 

| Preset   | cron equivalent |
|----------|-----------------|
| @hour    | 0 * * * *       | 
| @daily   | 0 0 * * *       |
| @weekly  | 0 0 * * 0       |
| @monthly | 0 0 1 * *       |
| @yearly  | 0 0 1 1 *       |

The @hourly preset means run once an hour at the beginning of the hour. It's equivalent to 0 asterisk asterisk asterisk asterisk in cron. The @daily, @weekly, @monthly, and @yearly presets behave similarly.

::: {.callout-caution collapse="true" apperance='simple'}
Note: "0 12 * * *" != "* 12 * * *"

"0 12 * * *" This cron expression specifies a specific minute and hour. It means the task should run at 12:00 PM (noon) every day. The 0 in the first position represents the minute, and 12 in the second position represents the hour.

"* 12 * * *" This cron expression uses an asterisk in the minute position, which means the task will run every minute of the 12th hour (noon) of every day. In other words, it will run once every minute between 12:00 PM and 12:59 PM.
:::

## Special presets

Airflow also has two special presets for schedule intervals. 

- @none means don't ever schedule the DAG and is used for manually triggered workflows.
- @once means to only schedule a DAG once.


::: {.callout-caution collapse="true" apperance='simple'}
Scheduling DAGs has an important nuance to consider. When scheduling DAG runs, Airflow will use the start date as the earliest possible value, but not actually schedule anything until at least one schedule interval has passed beyond the start date. Given a start_date of February 25, 2020 and a @daily schedule interval, Airflow would then use the date of February 26, 2020 for the first run of the DAG. This can be tricky to consider when adding new DAG schedules, especially if they have longer schedule intervals.
:::

## Exercises